# Machine Learning Operations (MLOps) - Part 2

![](../images/containers.jpeg)

- [Docker](https://docs.docker.com/get-docker/) is THE MOST popular way to package code and dependencies for web applications including ML model deployments
- Docker works on the concept of [Containers](https://www.docker.com/resources/what-container) which is a self-contained unit of OS, dependencies and code necessary to run an App
- Docker containers are EVERYWHERE! Chances are, every single modern app or website you'll ever interact with is running in a Docker container
- Despite being ubiquitous everywhere, it's surprisingly easy to create a docker container of your Flask App, if you stick to the basics!
- Of course, Docker has significant depth that you can exploit to build all kinds of complex applications!
- In this lesson, we'll use Docker to containerize our ML model flask app and run it both locally and in the cloud!
- We start from where we left off in the previous lesson. We already have a serve.py file that can run our model as a Flask API
- Steps
    1. Train an ML model as per normal
    1. Save the model as per normal
    1. Test the saved model
    1. Create a `serve.py` file to serve the saved model as a Flask API
    1. Test the `serve.py` file
    1. [NEW] Create a Docker container to package our serving code
    1. [NEW] Test the Docker container

# Docker
![](../images/docker_logo.png)
- A docker container is a mini-computer running within your computer
- Docker allows us to create ***"Containerized Applications"***
- It's smaller and more efficient than using a brand new VM for each application
- Take a look at this [link](https://www.docker.com/resources/what-container) to read up more about docker containers
![](../images/container_vs_VM.png)

## 6. [NEW] Create a Docker container to package our serving code

There are a few steps involved
1. Create a `requirements.txt` file including all the Python libraries we need for the app to work
1. Create a `Dockerfile` which contains the steps on how to create the Docker container

In [1]:
# Check the versions of each library that is needed by our serve.py
import numpy as np
import flask
import tensorflow as tf

print(f'Numpy version: {np.__version__}')
print(f'Flask version: {flask.__version__}')
print(f'Tensorflow version: {tf.__version__}')

Numpy version: 1.22.2
Numpy version: 2.0.3
Numpy version: 2.8.0


### 6.1 Create a requirements.txt file including all the Python libraries we need for the app to work
Create a `requirements.txt` file with the below contents:

```
numpy==1.22.2
flask==2.0.3
tensorflow==2.8.0
```

### 6.2 Create a Dockerfile which contains the steps on how to create the Docker container
- Create a `Dockerfile` (no file extension!) file with the below contents
- For a complete list of all possible Dockerfile commands, see [here](https://docs.docker.com/engine/reference/builder/)
- Below are probably the most common commands you'll ever need

```
# Use the official lightweight Python image.
# https://hub.docker.com/_/python
FROM python:3.8-slim

# Copy all the files needed for the app to work
COPY requirements.txt serve.py cats_vs_dogs.h5 .

# Install all the necessary libraries
RUN pip install -r requirements.txt

# Run the app!
CMD python serve.py
```

## 7. [NEW] Test the Docker container
There are many many ways to run a docker container

### 7.1. Locally or any server that has installed docker. Run on Terminal: 

- Build the docker container from the Dockerfile in the current directory(.) 
- Give it a name given by --tag
- Docker naming convention is <name_of_service>:<version>. latest means latest version
    
```
docker build . --tag cats_v_dogs:latest
```

In [2]:
# Run the docker container in -d (detached) mode
## -p 5000:5000 -> map the port 5000 in the docker container to 5000 on your local machine
## --rm -> remove the container when its stopped (optional)
## --name cats_v_dogs -> name the running container cats_v_dogs
## cats_v_dogs:latest -> the name of the container to run

! docker run -d -p 8080:8080 --rm --name cats_v_dogs cats_v_dogs:latest 

38275df4d27d7e34462c23c7e390c83730593ddf4279835608c2785588c30ddb


In [3]:
# Test the docker locally
import requests
from tensorflow.keras.preprocessing import image

In [4]:
# Load the images into Python
images = ['../images/test.jpeg', '../images/test_1.jpeg']
test_images = [image.load_img(img, target_size=(224, 224)) for img in images]

# Convert the images to a matrix of numbers
test_images = [image.img_to_array(img).tolist() for img in test_images]

In [5]:
user_input = {'X': test_images}

# Note the IP address when running docker on your local is the same as running the code directly on the terminal!
response = requests.post('http://127.0.0.1:8080/predict', json=user_input)
response.json()

{'response': ['Dog', 'Cat']}

In [6]:
# To check all running and stopped docker containers
! docker ps --all

CONTAINER ID   IMAGE                COMMAND                  CREATED          STATUS         PORTS                    NAMES
38275df4d27d   cats_v_dogs:latest   "/bin/sh -c 'python …"   11 seconds ago   Up 9 seconds   0.0.0.0:8080->8080/tcp   cats_v_dogs


In [7]:
# To stop the running docker container using its name
! docker stop cats_v_dogs

cats_v_dogs


### 7.2 To GCP Cloud Run
- Remember to install and init [gcloud CLI](https://cloud.google.com/sdk/docs/install) before running this code

```
gcloud run deploy cats-v-dogs --source . --region us-west1
```

- Get the IP address from the URL parameter
![](../images/GCP_cloud_run.png)

- GCP Cloud run by default only allocates 512MiB Memory to your app. This is mostly not sufficient for Tensorflow models, so you can edit the deployed app to increase the Memory to 1Gib

![](../images/GCP_cloud_run_edit.png)
![](../images/GCP_cloud_run_edit_2.png)

- GCP Cloud run is incredibly cheap, even [free for modest usage](https://cloud.google.com/products/calculator#id=b6498bda-2e9a-4d14-adb6-bf3915c7812f). 
- Still if you want to clean up all the GCP resources you created so that you dont pay anything. You need to delete the resources from the below 2 GCP services
    1. Cloud Run
    1. Artifact Registry (stores the docker image on GCP). Look inside `cloud-run-source-deploy` directory
    1. Cloud Storage. Delete the cloud storage bucket ending with `_cloudbuild`

In [14]:
# Note the IP address when running the model in Google Cloud Run is the URL parameter
response = requests.post('https://cats-v-dogs-a4rmk57awq-uw.a.run.app/predict', json=user_input)
response.json()

{'response': ['Dog', 'Cat']}

### 7.3 To AWS Fargate 
- Follow the very well written instructions from [this Medium Post](https://towardsdatascience.com/deploy-pycaret-and-streamlit-app-using-aws-fargate-serverless-infrastructure-8b7d7c0584c2#36ea)

### 7.4 To a Kubernetes (K8S) Cluster
- [Kubernetes](https://kubernetes.io/) is one of the MOST scalable way to deploy a Docker container in production. It can scale to support millions of requests per second. It's what's used in most big technology companies that have high internet traffic. But, managing a Kubernetes cluster is notoriously hard and requires a team of trained people to manage effectively.
- For most DS use cases, we don't need to worry about managing a K8S cluster. But if you even want to play around with K8S by deploying your app, you can use the below steps.
- Steps taken from GKE documentation

#### Deploy our Docker container to GKE
1. Search `Kubernetes Engine` on GCP
1. Click Workloads.
1. Click Deploy.
1. In the Edit container section, select Existing container image.
1. In the Image path field, click Select.
1. In the Select container image pane, select the flask app image we created for Cloud Run.
1. Click Select.
1. Click Done.
1. In the Environment variables click Add Environment Variable and enter the below values
    - Key 1: PORT
    - Value 1: 8080
1. Click Continue.
1. In the Configuration section, 
    - Update the Application name
    - under Labels, enter app for Key and cats-v-dogs for Value.
1. Under the Configuration section, click the View YAML button under Configuration YAML. This opens a YAML configuration file representing the two Kubernetes API resources about to be deployed into your cluster. Nothing to do here.
1. Click Close.
1. Click Deploy. When the Deployment Pods are ready, the Deployment details page opens.
1. Under Managed pods, note the three running Pods for the cats-v-dogs Deployment.

#### Expose a sample app to the internet
1. Click Workloads.
1. Click on cats-v-dogs in the Name column.
1. From the Deployment details page, click the Actions > Expose button.
1. In the Expose dialog, under the Port Mapping section, set
    - Target port to 8080.
    - Service type drop-down list to Load balancer.
1. Click Expose to create a Kubernetes Service for cats-v-dogs. When the Load Balancer is ready, the Service details page opens.
1. Copy the External endpoints address

🎉 Success
You have successfully deployed a containerized web application to a K8S cluster on GCP!

#### Cleanup
1. Delete the Service from Services & Ingress
1. Delete the cluster from Clusters
1. Delete your container image from Artifact Registry

# Conclusion
- You now have packaged your model as a Docker container and deployed as a real time endpoint on your local machine, GCP Cloud Run and AWS Fargate!
- Using Docker is one of THE BEST ways to "deploy" a model
- Pros:
    - Each model is self-contained with all its dependencies and no chance of conflicts
    - Very robust cloud services like GCP Cloud Run and AWS Fargate that allow you to deploy the model in a "serverless" fashion. [Serverless](https://en.wikipedia.org/wiki/Serverless_computing) is all the rage these days as you only need to take care of the code and the cloud provider will handle all the hardware without any hassle for you!
    - No more "It works on my machine cliche"! Docker ensures you build once and run anywhere!
- Cons:
    - Not much except you need to learn Docker and Linux commands
    - And a bit difficult to debug when building more complex applications. Though ML model deployments in Docker is relatively straightforward

# Test your knowledge
1. Try to package the ham vs spam TFIDF + SklearnNLP classifier model from [5.06-lesson-nlp-ii](../../5.06-lesson-nlp-ii/solution-code/solution-code.ipynb) as a Flask API with Docker.
1. Deploy the app on GCP Cloud Run
1. Easily sell access to your API on [RapidAPI](https://rapidapi.com/)!!! Check out [Leo's Name-Gender prediction API](https://rapidapi.com/stephenleo87-DGFI1at-XQ/api/name-gender1/)

# Easy Steps to become a Millionaire
1. Invent a fancy API
1. Wrap it into a Docker container
1. Host it in GCP Cloud Run
1. Publish it on Rapid API 
1. Become a Millionaire!

![](../images/make-it-rain-meme.jpeg)